<li>Details - Duration 20 minutes
<ul>
<li>Data is available in HDFS /public/randomtextwriter</li>
<li>Get word count for the input data using space as delimiter (for each word, we need to get how many types it is repeated in the entire input data set)</li>
<li>Number of executors should be 10</li>
<li>Executor memory should be 3 GB</li>
<li>Executor cores should be 20 in total (2 per executor)</li>
<li>Number of output files should be 8</li>
<li>Avro dependency details: groupId -&gt; com.databricks, artifactId -&gt; spark-avro_2.10, version -&gt; 2.0.1</li>
<li>Target Directory: /user/&lt;YOUR_USER_ID&gt;/solutions/solution05/wordcount</li>
<li>Target File Format: Avro</li>
<li>Target fields: word, count</li>
<li>Compression: N/A or default</li>
</ul>
</li>

In [8]:
! hdfs dfs -cat /user/pi/in/word_count.text

2020-06-09 19:18:17,706 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-06-09 19:18:20,993 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
The history of New York begins around 10,000 BC, when the first Native Americans arrived. By 1100 AD, New York's main native cultures, the Iroquoian and Algonquian, had developed. European discovery of New York was led by the French in 1524 and the first land claim came in 1609 by the Dutch. As part of New Netherland, the colony was important in the fur trade and eventually became an agricultural resource thanks to the patroon system. In 1626 the Dutch bought the island of Manhattan from Native Americans.[1] In 1664, England renamed the colony New York, after the Duke of York (later James II & VII.) New York City gained prominence in the 18th century as a major trading port in the Thirteen Colonies

### run pyspark console as below
***

yspark --master spark://192.168.1.109:7077 --num-executors 4 --executor-memory 2GB --executor-cores 2 --packages org.apache.spark:spark-avro_2.11:2.4.0

***

In [4]:
textRawRDD = sc.textFile("/user/pi/in/word_count.text")
textRawRDD.take(10)

["The history of New York begins around 10,000 BC, when the first Native Americans arrived. By 1100 AD, New York's main native cultures, the Iroquoian and Algonquian, had developed. European discovery of New York was led by the French in 1524 and the first land claim came in 1609 by the Dutch. As part of New Netherland, the colony was important in the fur trade and eventually became an agricultural resource thanks to the patroon system. In 1626 the Dutch bought the island of Manhattan from Native Americans.[1] In 1664, England renamed the colony New York, after the Duke of York (later James II & VII.) New York City gained prominence in the 18th century as a major trading port in the Thirteen Colonies.",
 '',
 "New York played a pivotal role during the American Revolution and subsequent war. The Stamp Act Congress in 1765 brought together representatives from across the Thirteen Colonies to form a unified response to British policies. The Sons of Liberty were active in New York City to 

In [6]:
words = textRawRDD.flatMap(lambda w:w.split(" ")).map(lambda w:(w,1))
words.take(10)

[('The', 1),
 ('history', 1),
 ('of', 1),
 ('New', 1),
 ('York', 1),
 ('begins', 1),
 ('around', 1),
 ('10,000', 1),
 ('BC,', 1),
 ('when', 1)]

In [7]:
from pyspark.sql import Row
wc = words.reduceByKey(lambda x,y:x+y).map(lambda r:Row(word=r[0],count=int(r[1])))
wc.take(10)

[Row(count=10, word='The'),
 Row(count=33, word='of'),
 Row(count=20, word='New'),
 Row(count=1, word='begins'),
 Row(count=4, word='around'),
 Row(count=1, word='BC,'),
 Row(count=1, word='when'),
 Row(count=1, word='Americans'),
 Row(count=1, word='1100'),
 Row(count=1, word='native')]

In [8]:
wcDF = wc.toDF()
wcDF.show()

+-----+---------+
|count|     word|
+-----+---------+
|   10|      The|
|   33|       of|
|   20|      New|
|    1|   begins|
|    4|   around|
|    1|      BC,|
|    1|     when|
|    1|Americans|
|    1|     1100|
|    1|   native|
|    1|cultures,|
|    1|Iroquoian|
|    2| European|
|    8|      was|
|    2|      led|
|   21|       in|
|    1|     1524|
|    1|    claim|
|    2|     came|
|    1|   Dutch.|
+-----+---------+
only showing top 20 rows



In [9]:
wcDF.printSchema()

root
 |-- count: long (nullable = true)
 |-- word: string (nullable = true)



In [10]:
wcDF.select('word','count').coalesce(8). \
write.format('com.databricks.spark.avro'). \
save('/user/pi/solutions/solution05/wordcount')

In [12]:
spark.read.format('com.databricks.spark.avro').load('/user/pi/solutions/solution05/wordcount').show()

+-----------+-----+
|       word|count|
+-----------+-----+
|    history|    1|
|       York|   17|
|     10,000|    1|
|        the|   71|
|      first|    5|
|     Native|    2|
|   arrived.|    1|
|         By|    1|
|        AD,|    1|
|     York's|    2|
|       main|    2|
|        and|   21|
|Algonquian,|    1|
|        had|    1|
| developed.|    1|
|  discovery|    1|
|         by|    2|
|     French|    1|
|       land|    1|
|       1609|    1|
+-----------+-----+
only showing top 20 rows



In [13]:
! hdfs dfs -ls /user/pi/solutions/solution05/wordcount/*

2020-06-09 20:01:23,886 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
-rw-r--r--   2 pi supergroup          0 2020-06-09 19:59 /user/pi/solutions/solution05/wordcount/_SUCCESS
-rw-r--r--   2 pi supergroup       1705 2020-06-09 19:59 /user/pi/solutions/solution05/wordcount/part-00000-cba9837d-046a-448c-991f-c8f8c03bc501-c000.avro
-rw-r--r--   2 pi supergroup       1941 2020-06-09 19:59 /user/pi/solutions/solution05/wordcount/part-00001-cba9837d-046a-448c-991f-c8f8c03bc501-c000.avro
